In [25]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from pandas.api.types import CategoricalDtype
import re

In [39]:
def parse_interval_column(group_series):
    """
    Parses bin strings like '[80.0, 90.0)' into an ordered categorical dtype,
    sorted by numeric left endpoint, suitable for saving and plotting.

    Parameters:
        group_series (pd.Series): A column of strings like '[80.0, 90.0)'

    Returns:
        pd.Series: An ordered categorical series with consistent sorting
    """

    # Drop missing values and extract unique labels
    group_labels = group_series.dropna().unique()

    # Sort labels by numeric left endpoint
    def get_left_edge(label):
        match = re.match(r"\[(\d+(?:\.\d+)?),", label)
        return float(match.group(1)) if match else float('inf')


    sorted_labels = sorted(group_labels, key=get_left_edge)


    # Create ordered categorical dtype
    cat_type = CategoricalDtype(categories=sorted_labels, ordered=True)

    # Apply it to the full series (including NaNs if present)
    return group_series.astype(cat_type)


1. codigos escolas distritos

In [68]:
# Load the Excel file
df = pd.read_excel("data_xlsx/codigos_escolas_distritos.xlsx", engine="openpyxl")

# Display the first few rows
print(df.head())

# see type of variables
print(df.dtypes)

# change floats to integers
df["escola_distrito"] = pd.to_numeric(df["escola_distrito"], errors="coerce").astype("Int64")
df["escola_concelho"] = pd.to_numeric(df["escola_concelho"], errors="coerce").astype("Int64")

has_leading_zeros = df['escola'].astype(str).str.match('^0+\d+')
if has_leading_zeros.any():
    print("Found values with leading zeros")
    print(df[has_leading_zeros]['your_column'])

has_leading_zeros = df['escola_distrito'].astype(str).str.match('^0+\d+')
if has_leading_zeros.any():
    print("Found values with leading zeros")
    print(df[has_leading_zeros]['your_column'])

has_leading_zeros = df['escola_concelho'].astype(str).str.match('^0+\d+')
if has_leading_zeros.any():
    print("Found values with leading zeros")
    print(df[has_leading_zeros]['your_column'])

df["escola"] = df["escola"].astype("category")
df["escola_descr"] = df["escola_descr"].astype("category")
df["escola_distrito"] = df["escola_distrito"].astype("category")
df["escola_distrito_descr"] = df["escola_distrito_descr"].astype("category")
df["escola_concelho"] = df["escola_concelho"].astype("category")
df["escola_concelho_descr"] = df["escola_concelho_descr"].astype("category")
df["escola_pubpriv"] = df["escola_pubpriv"].astype("category")

# see type of variables
print(df.dtypes)

# Total rows
total_rows = len(df)

# Rows with at least one NaN
rows_with_nan = df.isna().any(axis=1).sum()

print(f"Total rows: {total_rows}")
print(f"Rows with at least one NaN: {rows_with_nan}")
print(f"Percentage with NaN: {(rows_with_nan/total_rows*100):.1f}%")

   escola                                       escola_descr  escola_distrito  \
0       2  Escola BÃ¡sica e SecundÃ¡ria de Michel Giacome...             15.0   
1      31  Escola BÃ¡sica e SecundÃ¡ria Dr. JoÃ£o Brito C...              2.0   
2      39            Escola BÃ¡sica e SecundÃ¡ria de Ourique              2.0   
3      47  Escola BÃ¡sica e SecundÃ¡ria de Cabeceiras de ...              3.0   
4      49  Escola BÃ¡sica e SecundÃ¡ria de Celorico de Basto              3.0   

  escola_distrito_descr  escola_concelho escola_concelho_descr escola_pubpriv  
0              SetÃºbal             11.0              Sesimbra            PUB  
1                  Beja              2.0            AlmodÃ´var            PUB  
2                  Beja             12.0               Ourique            PUB  
3                 Braga              4.0   Cabeceiras de Basto            PUB  
4                 Braga              5.0     Celorico de Basto            PUB  
escola                     int64


In [7]:
df.to_parquet("data_parquet/codigos_escolas_distritos.parquet", engine="pyarrow")  # Save

2. inflacao disciplinas

In [70]:
dfs = pd.read_excel("data_xlsx/inflacao_disciplina.xlsx", sheet_name=None, engine="openpyxl")

# Get the number of sheets
num_sheets = len(dfs)
print(f"Number of sheets: {num_sheets}")

# Show the head of each sheet
for sheet_name, df in dfs.items():
    print(f"\nHead of sheet '{sheet_name}':")
    print(df.head())  # You can adjust the number of rows shown with head(n)

df = dfs["Folha2"]

# see type of variables
print(df.dtypes)

has_leading_zeros = df['EXAME'].astype(str).str.match('^0+\d+')
if has_leading_zeros.any():
    print("Found values with leading zeros")
    print(df[has_leading_zeros]['EXAME'])


Number of sheets: 2

Head of sheet 'Folha1':
Empty DataFrame
Columns: []
Index: []

Head of sheet 'Folha2':
       GROUP PUBPRIV  EXAME  INFLACAO    INF_STD  NSAMPLES  \
0    [60,70)     PUB    126    11.750  54.328476         4   
1   [90,100)     PUB    126    23.000  28.011902         4   
2  [100,110)     PUB    126    16.600  21.972710         5   
3  [110,120)     PUB    126     7.750  18.355290         4   
4  [100,110)     PUB    127    -8.125  42.265952         8   

                  EXAME_DESCR  
0  Geometria Descritiva A -NE  
1  Geometria Descritiva A -NE  
2  Geometria Descritiva A -NE  
3  Geometria Descritiva A -NE  
4   Literatura Portuguesa -NE  
GROUP           object
PUBPRIV         object
EXAME            int64
INFLACAO       float64
INF_STD        float64
NSAMPLES         int64
EXAME_DESCR     object
dtype: object


In [71]:
df['GROUP'] = pd.IntervalIndex.from_arrays(
    df['GROUP'].str.extract('\[(\d+),(\d+)\)').astype(float).iloc[:, 0],
    df['GROUP'].str.extract('\[(\d+),(\d+)\)').astype(float).iloc[:, 1],
    closed='left'
)

In [72]:
print(df.dtypes)
print(df.head())

GROUP          interval[float64, left]
PUBPRIV                         object
EXAME                            int64
INFLACAO                       float64
INF_STD                        float64
NSAMPLES                         int64
EXAME_DESCR                     object
dtype: object
            GROUP PUBPRIV  EXAME  INFLACAO    INF_STD  NSAMPLES  \
0    [60.0, 70.0)     PUB    126    11.750  54.328476         4   
1   [90.0, 100.0)     PUB    126    23.000  28.011902         4   
2  [100.0, 110.0)     PUB    126    16.600  21.972710         5   
3  [110.0, 120.0)     PUB    126     7.750  18.355290         4   
4  [100.0, 110.0)     PUB    127    -8.125  42.265952         8   

                  EXAME_DESCR  
0  Geometria Descritiva A -NE  
1  Geometria Descritiva A -NE  
2  Geometria Descritiva A -NE  
3  Geometria Descritiva A -NE  
4   Literatura Portuguesa -NE  


In [73]:
#change to categories for efficiency and correctness
df["PUBPRIV"] = df["PUBPRIV"].astype("category")
df["EXAME"] = df["EXAME"].astype("category")
df["EXAME_DESCR"] = df["EXAME_DESCR"].astype("category")

# see type of variables
print(df.dtypes)
print(df.head())

# Total rows
total_rows = len(df)

# Rows with at least one NaN
rows_with_nan = df.isna().any(axis=1).sum()

print(f"Total rows: {total_rows}")
print(f"Rows with at least one NaN: {rows_with_nan}")
print(f"Percentage with NaN: {(rows_with_nan/total_rows*100):.1f}%")

GROUP          interval[float64, left]
PUBPRIV                       category
EXAME                         category
INFLACAO                       float64
INF_STD                        float64
NSAMPLES                         int64
EXAME_DESCR                   category
dtype: object
            GROUP PUBPRIV EXAME  INFLACAO    INF_STD  NSAMPLES  \
0    [60.0, 70.0)     PUB   126    11.750  54.328476         4   
1   [90.0, 100.0)     PUB   126    23.000  28.011902         4   
2  [100.0, 110.0)     PUB   126    16.600  21.972710         5   
3  [110.0, 120.0)     PUB   126     7.750  18.355290         4   
4  [100.0, 110.0)     PUB   127    -8.125  42.265952         8   

                  EXAME_DESCR  
0  Geometria Descritiva A -NE  
1  Geometria Descritiva A -NE  
2  Geometria Descritiva A -NE  
3  Geometria Descritiva A -NE  
4   Literatura Portuguesa -NE  
Total rows: 887
Rows with at least one NaN: 40
Percentage with NaN: 4.5%


In [22]:


df.to_parquet("data_parquet/inflacao_disciplina.parquet", engine="pyarrow")  # Save

3. inflacao distritos

In [74]:
dfs = pd.read_excel("data_xlsx/inflacao_distritos.xlsx", sheet_name=None, engine="openpyxl")

# Get the number of sheets
num_sheets = len(dfs)
print(f"Number of sheets: {num_sheets}")

# Show the head of each sheet
for sheet_name, df in dfs.items():
    print(f"\nHead of sheet '{sheet_name}':")
    print(df.head())  # You can adjust the number of rows shown with head(n)

df = dfs["Folha2"]

# see type of variables
print(df.dtypes)

df["ESCOLA_DISTRITO"] = pd.to_numeric(df["ESCOLA_DISTRITO"], errors="coerce").astype("Int64")

has_leading_zeros = df['ESCOLA_DISTRITO'].astype(str).str.match('^0+\d+')
if has_leading_zeros.any():
    print("Found values with leading zeros")
    print(df[has_leading_zeros]['EXAME'])

Number of sheets: 2

Head of sheet 'Folha1':
Empty DataFrame
Columns: []
Index: []

Head of sheet 'Folha2':
     GROUP PUBPRIV  ESCOLA_DISTRITO    INFLACAO    INF_STD  NSAMPLES  \
0   [0,10)     PRI              1.0  107.000000  30.670290         4   
1  [10,20)     PRI              1.0   90.111111   9.498538        18   
2  [20,30)     PRI              1.0   82.971429  15.951186        35   
3  [30,40)     PRI              1.0   77.961538  15.180773       130   
4  [40,50)     PRI              1.0   68.770161  14.884426       248   

  ESCOLA_DISTRITO_DESCR  
0                Aveiro  
1                Aveiro  
2                Aveiro  
3                Aveiro  
4                Aveiro  
GROUP                     object
PUBPRIV                   object
ESCOLA_DISTRITO          float64
INFLACAO                 float64
INF_STD                  float64
NSAMPLES                   int64
ESCOLA_DISTRITO_DESCR     object
dtype: object


In [75]:
df['GROUP'] = pd.IntervalIndex.from_arrays(
    df['GROUP'].str.extract('\[(\d+),(\d+)\)').astype(float).iloc[:, 0],
    df['GROUP'].str.extract('\[(\d+),(\d+)\)').astype(float).iloc[:, 1],
    closed='left'
)

#change to categories for efficiency and correctness
df["PUBPRIV"] = df["PUBPRIV"].astype("category")
df["ESCOLA_DISTRITO"] = df["ESCOLA_DISTRITO"].astype("category")
df["ESCOLA_DISTRITO_DESCR"] = df["ESCOLA_DISTRITO_DESCR"].astype("category")

# see type of variables
print(df.dtypes)
print(df.head())

# Total rows
total_rows = len(df)

# Rows with at least one NaN
rows_with_nan = df.isna().any(axis=1).sum()

print(f"Total rows: {total_rows}")
print(f"Rows with at least one NaN: {rows_with_nan}")
print(f"Percentage with NaN: {(rows_with_nan/total_rows*100):.1f}%")

GROUP                    interval[float64, left]
PUBPRIV                                 category
ESCOLA_DISTRITO                         category
INFLACAO                                 float64
INF_STD                                  float64
NSAMPLES                                   int64
ESCOLA_DISTRITO_DESCR                   category
dtype: object
          GROUP PUBPRIV ESCOLA_DISTRITO    INFLACAO    INF_STD  NSAMPLES  \
0   [0.0, 10.0)     PRI               1  107.000000  30.670290         4   
1  [10.0, 20.0)     PRI               1   90.111111   9.498538        18   
2  [20.0, 30.0)     PRI               1   82.971429  15.951186        35   
3  [30.0, 40.0)     PRI               1   77.961538  15.180773       130   
4  [40.0, 50.0)     PRI               1   68.770161  14.884426       248   

  ESCOLA_DISTRITO_DESCR  
0                Aveiro  
1                Aveiro  
2                Aveiro  
3                Aveiro  
4                Aveiro  
Total rows: 785
Rows with at l

In [25]:
df.to_parquet("data_parquet/inflacao_distritos.parquet", engine="pyarrow")  # Save

4. inflacao_escola_bins_anos

In [42]:
dfs = pd.read_excel("data_xlsx/inflacao_escola_bins_anos.xlsx", sheet_name=None, engine="openpyxl")

# Get the number of sheets
num_sheets = len(dfs)
print(f"Number of sheets: {num_sheets}")

# Show the head of each sheet
for sheet_name, df in dfs.items():
    print(f"\nHead of sheet '{sheet_name}':")
    print(df.head())  # You can adjust the number of rows shown with head(n)

df = dfs["Folha2"]

# see type of variables
print(df.dtypes)

has_leading_zeros = df['ESCOLA'].astype(str).str.match('^0+\d+')
if has_leading_zeros.any():
    print("Found values with leading zeros")
    print(df[has_leading_zeros]['ESCOLA'])

Number of sheets: 2

Head of sheet 'Folha1':
Empty DataFrame
Columns: []
Index: []

Head of sheet 'Folha2':
     GROUP  ESCOLA   ANO   INFLACAO    INF_STD  NSAMPLES  \
0  [30,40)       2  2014  73.750000   4.924429         4   
1  [40,50)       2  2014  62.625000   4.657943         8   
2  [50,60)       2  2014  55.000000  11.520732        12   
3  [60,70)       2  2014  46.333333  12.804389        15   
4  [70,80)       2  2014  40.222222  12.587472         9   

                                        ESCOLA_DESCR ESCOLA_PUBPRIV  
0  Escola BÃ¡sica e SecundÃ¡ria de Michel Giacometti            PUB  
1  Escola BÃ¡sica e SecundÃ¡ria de Michel Giacometti            PUB  
2  Escola BÃ¡sica e SecundÃ¡ria de Michel Giacometti            PUB  
3  Escola BÃ¡sica e SecundÃ¡ria de Michel Giacometti            PUB  
4  Escola BÃ¡sica e SecundÃ¡ria de Michel Giacometti            PUB  
GROUP              object
ESCOLA              int64
ANO                 int64
INFLACAO          float64
INF_STD

In [43]:
df['GROUP'] = parse_interval_column(df['GROUP'])

#change to categories for efficiency and correctness
df["ESCOLA"] = df["ESCOLA"].astype("category")
df["ESCOLA_DESCR"] = df["ESCOLA_DESCR"].astype("category")
df["ESCOLA_PUBPRIV"] = df["ESCOLA_PUBPRIV"].astype("category")

# see type of variables
print(df.dtypes)
print(df.head())

# Total rows
total_rows = len(df)

# Rows with at least one NaN
rows_with_nan = df.isna().any(axis=1).sum()

print(f"Total rows: {total_rows}")
print(f"Rows with at least one NaN: {rows_with_nan}")
print(f"Percentage with NaN: {(rows_with_nan/total_rows*100):.1f}%")

GROUP             category
ESCOLA            category
ANO                  int64
INFLACAO           float64
INF_STD            float64
NSAMPLES             int64
ESCOLA_DESCR      category
ESCOLA_PUBPRIV    category
dtype: object
     GROUP ESCOLA   ANO   INFLACAO    INF_STD  NSAMPLES  \
0  [30,40)      2  2014  73.750000   4.924429         4   
1  [40,50)      2  2014  62.625000   4.657943         8   
2  [50,60)      2  2014  55.000000  11.520732        12   
3  [60,70)      2  2014  46.333333  12.804389        15   
4  [70,80)      2  2014  40.222222  12.587472         9   

                                        ESCOLA_DESCR ESCOLA_PUBPRIV  
0  Escola BÃ¡sica e SecundÃ¡ria de Michel Giacometti            PUB  
1  Escola BÃ¡sica e SecundÃ¡ria de Michel Giacometti            PUB  
2  Escola BÃ¡sica e SecundÃ¡ria de Michel Giacometti            PUB  
3  Escola BÃ¡sica e SecundÃ¡ria de Michel Giacometti            PUB  
4  Escola BÃ¡sica e SecundÃ¡ria de Michel Giacometti            P

In [49]:
df.to_parquet("data_parquet/inflacao_escola_bins_anos.parquet", engine="pyarrow")  # Save

5. inflacao_escola_bins

In [40]:
dfs = pd.read_excel("data_xlsx/inflacao_escola_bins.xlsx", sheet_name=None, engine="openpyxl")

# Get the number of sheets
num_sheets = len(dfs)
print(f"Number of sheets: {num_sheets}")

# Show the head of each sheet
for sheet_name, df in dfs.items():
    print(f"\nHead of sheet '{sheet_name}':")
    print(df.head())  # You can adjust the number of rows shown with head(n)

df = dfs["Folha2"]

# see type of variables
print(df.dtypes)

has_leading_zeros = df['ESCOLA'].astype(str).str.match('^0+\d+')
if has_leading_zeros.any():
    print("Found values with leading zeros")
    print(df[has_leading_zeros]['ESCOLA'])

Number of sheets: 2

Head of sheet 'Folha1':
Empty DataFrame
Columns: []
Index: []

Head of sheet 'Folha2':
     GROUP  ESCOLA   INFLACAO    INF_STD  NSAMPLES  \
0  [10,20)       2  93.500000  11.096975         8   
1  [20,30)       2  81.357143   8.983807        14   
2  [30,40)       2  78.139535   9.946479        43   
3  [40,50)       2  66.636364  12.363266        66   
4  [50,60)       2  62.225806  14.526029       124   

                                        ESCOLA_DESCR ESCOLA_PUBPRIV  
0  Escola BÃ¡sica e SecundÃ¡ria de Michel Giacometti            PUB  
1  Escola BÃ¡sica e SecundÃ¡ria de Michel Giacometti            PUB  
2  Escola BÃ¡sica e SecundÃ¡ria de Michel Giacometti            PUB  
3  Escola BÃ¡sica e SecundÃ¡ria de Michel Giacometti            PUB  
4  Escola BÃ¡sica e SecundÃ¡ria de Michel Giacometti            PUB  
GROUP              object
ESCOLA              int64
INFLACAO          float64
INF_STD           float64
NSAMPLES            int64
ESCOLA_DESCR     

In [41]:
df['GROUP'] = parse_interval_column(df['GROUP'])
parsed = df['GROUP']
print(parsed.head())
print(parsed.cat.categories)


#change to categories for efficiency and correctness
df["ESCOLA"] = df["ESCOLA"].astype("category")
df["ESCOLA_DESCR"] = df["ESCOLA_DESCR"].astype("category")
df["ESCOLA_PUBPRIV"] = df["ESCOLA_PUBPRIV"].astype("category")

# see type of variables
print(df.dtypes)
print(df.head())

# Total rows
total_rows = len(df)

# Rows with at least one NaN
rows_with_nan = df.isna().any(axis=1).sum()

print(f"Total rows: {total_rows}")
print(f"Rows with at least one NaN: {rows_with_nan}")
print(f"Percentage with NaN: {(rows_with_nan/total_rows*100):.1f}%")

0    [10,20)
1    [20,30)
2    [30,40)
3    [40,50)
4    [50,60)
Name: GROUP, dtype: category
Categories (20, object): ['[0,10)' < '[10,20)' < '[20,30)' < '[30,40)' ... '[160,170)' < '[170,180)' < '[180,190)' < '[190,200]']
Index(['[0,10)', '[10,20)', '[20,30)', '[30,40)', '[40,50)', '[50,60)',
       '[60,70)', '[70,80)', '[80,90)', '[90,100)', '[100,110)', '[110,120)',
       '[120,130)', '[130,140)', '[140,150)', '[150,160)', '[160,170)',
       '[170,180)', '[180,190)', '[190,200]'],
      dtype='object')
GROUP             category
ESCOLA            category
INFLACAO           float64
INF_STD            float64
NSAMPLES             int64
ESCOLA_DESCR      category
ESCOLA_PUBPRIV    category
dtype: object
     GROUP ESCOLA   INFLACAO    INF_STD  NSAMPLES  \
0  [10,20)      2  93.500000  11.096975         8   
1  [20,30)      2  81.357143   8.983807        14   
2  [30,40)      2  78.139535   9.946479        43   
3  [40,50)      2  66.636364  12.363266        66   
4  [50,60)      2

In [32]:
df.to_parquet("data_parquet/inflacao_escola_bins.parquet", engine="pyarrow")  # Save

6. inflacao_escolas_anos

In [80]:
dfs = pd.read_excel("data_xlsx/inflacao_escolas_anos.xlsx", sheet_name=None, engine="openpyxl")

# Get the number of sheets
num_sheets = len(dfs)
print(f"Number of sheets: {num_sheets}")

# Show the head of each sheet
for sheet_name, df in dfs.items():
    print(f"\nHead of sheet '{sheet_name}':")
    print(df.head())  # You can adjust the number of rows shown with head(n)

df = dfs["Folha2"]

# see type of variables
print(df.dtypes)

has_leading_zeros = df['ESCOLA'].astype(str).str.match('^0+\d+')
if has_leading_zeros.any():
    print("Found values with leading zeros")
    print(df[has_leading_zeros]['ESCOLA'])

Number of sheets: 2

Head of sheet 'Folha1':
Empty DataFrame
Columns: []
Index: []

Head of sheet 'Folha2':
   ESCOLA   ANO   INFLACAO    INF_STD  NSAMPLES  \
0       2  2014  25.248677  26.476335       189   
1      39  2014  35.550000  41.098350        40   
2      49  2014  33.449057  25.026165       265   
3      57  2014  28.392857  33.254729        28   
4      62  2014  45.120000  32.764241        50   

                                        ESCOLA_DESCR ESCOLA_PUBPRIV  
0  Escola BÃ¡sica e SecundÃ¡ria de Michel Giacometti            PUB  
1            Escola BÃ¡sica e SecundÃ¡ria de Ourique            PUB  
2  Escola BÃ¡sica e SecundÃ¡ria de Celorico de Basto            PUB  
3    Escola BÃ¡sica e SecundÃ¡ria de Terras de Bouro            PUB  
4   Escola BÃ¡sica e SecundÃ¡ria de Alfandega da FÃ©            PUB  
ESCOLA              int64
ANO                 int64
INFLACAO          float64
INF_STD           float64
NSAMPLES            int64
ESCOLA_DESCR       object
ESCOLA_PU

In [81]:
#change to categories for efficiency and correctness
df["ESCOLA"] = df["ESCOLA"].astype("category")
df["ESCOLA_DESCR"] = df["ESCOLA_DESCR"].astype("category")
df["ESCOLA_PUBPRIV"] = df["ESCOLA_PUBPRIV"].astype("category")

# see type of variables
print(df.dtypes)
print(df.head())

# Total rows
total_rows = len(df)

# Rows with at least one NaN
rows_with_nan = df.isna().any(axis=1).sum()

print(f"Total rows: {total_rows}")
print(f"Rows with at least one NaN: {rows_with_nan}")
print(f"Percentage with NaN: {(rows_with_nan/total_rows*100):.1f}%")

ESCOLA            category
ANO                  int64
INFLACAO           float64
INF_STD            float64
NSAMPLES             int64
ESCOLA_DESCR      category
ESCOLA_PUBPRIV    category
dtype: object
  ESCOLA   ANO   INFLACAO    INF_STD  NSAMPLES  \
0      2  2014  25.248677  26.476335       189   
1     39  2014  35.550000  41.098350        40   
2     49  2014  33.449057  25.026165       265   
3     57  2014  28.392857  33.254729        28   
4     62  2014  45.120000  32.764241        50   

                                        ESCOLA_DESCR ESCOLA_PUBPRIV  
0  Escola BÃ¡sica e SecundÃ¡ria de Michel Giacometti            PUB  
1            Escola BÃ¡sica e SecundÃ¡ria de Ourique            PUB  
2  Escola BÃ¡sica e SecundÃ¡ria de Celorico de Basto            PUB  
3    Escola BÃ¡sica e SecundÃ¡ria de Terras de Bouro            PUB  
4   Escola BÃ¡sica e SecundÃ¡ria de Alfandega da FÃ©            PUB  
Total rows: 4530
Rows with at least one NaN: 2
Percentage with NaN: 0.0%


In [55]:
df.to_parquet("data_parquet/inflacao_escolas_anos.parquet", engine="pyarrow")  # Save

7. inflacao_escolas

In [82]:
dfs = pd.read_excel("data_xlsx/inflacao_escolas.xlsx", sheet_name=None, engine="openpyxl")

# Get the number of sheets
num_sheets = len(dfs)
print(f"Number of sheets: {num_sheets}")

# Show the head of each sheet
for sheet_name, df in dfs.items():
    print(f"\nHead of sheet '{sheet_name}':")
    print(df.head())  # You can adjust the number of rows shown with head(n)

df = dfs["Folha2"]

# see type of variables
print(df.dtypes)

has_leading_zeros = df['ESCOLA'].astype(str).str.match('^0+\d+')
if has_leading_zeros.any():
    print("Found values with leading zeros")
    print(df[has_leading_zeros]['ESCOLA'])

Number of sheets: 2

Head of sheet 'Folha1':
Empty DataFrame
Columns: []
Index: []

Head of sheet 'Folha2':
   ESCOLA   INFLACAO    INF_STD  NSAMPLES  \
0       2  30.116519  26.657631      2034   
1      31  36.767654  29.290713       439   
2      39  31.858726  28.988398       361   
3      47  40.367316  28.125763       667   
4      49  34.724376  27.158311      1883   

                                        ESCOLA_DESCR ESCOLA_PUBPRIV  
0  Escola BÃ¡sica e SecundÃ¡ria de Michel Giacometti            PUB  
1  Escola BÃ¡sica e SecundÃ¡ria Dr. JoÃ£o de Brit...            PUB  
2            Escola BÃ¡sica e SecundÃ¡ria de Ourique            PUB  
3  Escola BÃ¡sica e SecundÃ¡ria de Cabeceiras de ...            PUB  
4  Escola BÃ¡sica e SecundÃ¡ria de Celorico de Basto            PUB  
ESCOLA              int64
INFLACAO          float64
INF_STD           float64
NSAMPLES            int64
ESCOLA_DESCR       object
ESCOLA_PUBPRIV     object
dtype: object


In [83]:
#change to categories for efficiency and correctness
df["ESCOLA"] = df["ESCOLA"].astype("category")
df["ESCOLA_DESCR"] = df["ESCOLA_DESCR"].astype("category")
df["ESCOLA_PUBPRIV"] = df["ESCOLA_PUBPRIV"].astype("category")

# see type of variables
print(df.dtypes)
print(df.head())

# Total rows
total_rows = len(df)

# Rows with at least one NaN
rows_with_nan = df.isna().any(axis=1).sum()

print(f"Total rows: {total_rows}")
print(f"Rows with at least one NaN: {rows_with_nan}")
print(f"Percentage with NaN: {(rows_with_nan/total_rows*100):.1f}%")

ESCOLA            category
INFLACAO           float64
INF_STD            float64
NSAMPLES             int64
ESCOLA_DESCR      category
ESCOLA_PUBPRIV    category
dtype: object
  ESCOLA   INFLACAO    INF_STD  NSAMPLES  \
0      2  30.116519  26.657631      2034   
1     31  36.767654  29.290713       439   
2     39  31.858726  28.988398       361   
3     47  40.367316  28.125763       667   
4     49  34.724376  27.158311      1883   

                                        ESCOLA_DESCR ESCOLA_PUBPRIV  
0  Escola BÃ¡sica e SecundÃ¡ria de Michel Giacometti            PUB  
1  Escola BÃ¡sica e SecundÃ¡ria Dr. JoÃ£o de Brit...            PUB  
2            Escola BÃ¡sica e SecundÃ¡ria de Ourique            PUB  
3  Escola BÃ¡sica e SecundÃ¡ria de Cabeceiras de ...            PUB  
4  Escola BÃ¡sica e SecundÃ¡ria de Celorico de Basto            PUB  
Total rows: 674
Rows with at least one NaN: 2
Percentage with NaN: 0.3%


In [60]:
df.to_parquet("data_parquet/inflacao_escolas.parquet", engine="pyarrow")  # Save